<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_6_salesbot_final_with_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 6 — Financial Analyst: Sales Prospecting Assistant (Tech-Focused)

**Цель:** Построить RAG-бота для работы с PDF-презентациями и письмами по продажам.

## 🔍 Что будет в ноутбуке
- Анализ структуры PDF-файлов по продажам
- Настройка разбиения текста (`RecursiveTextSplitter`) и векторного индекса
- Запросы к боту и оценка релевантности ответов


## 📁 Типы данных в продажах: зачем это важно для RAG
- **Презентации** содержат блоки: проблема, решение, выгоды, призыв к действию (CTA)
- **Письма** часто следуют структуре AIDA (attention → interest → desire → action)
- Бот должен извлекать **цитаты**, **выгоды**, **рекомендации** по нужному сегменту

## 🔧 Этапы построения RAG-бота
1. Загрузка PDF-файлов
2. Разбиение текста на чанки
3. Построение индекса
4. Ответ на вопрос → проверка: «ссылается ли бот на правильные фрагменты?»

In [1]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.1 MB/s eta 0:00:00


In [13]:
# Импорты
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from IPython.display import Markdown

In [14]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [15]:
# Загрузка PDF-документов из папки 'data'
docs = SimpleDirectoryReader("data").load_data()

🧰 Что делает SentenceSplitter:
Разбивает текст на логические чанки (блоки по ~500–1000 токенов)

Делает это рекурсивно по структуре текста: сначала по параграфам → потом по предложениям → потом по словам

Поддерживает overlap (перекрытие), чтобы сохранить контекст между фрагментами

🎯 Зачем это важно:
Каждый чанк становится отдельным Node в LlamaIndex

Векторное представление (embeddings) создаётся по каждому чанку отдельно

При запросе бот находит наиболее релевантные чанки, а не весь документ

🔹 Если не использовать сплиттер — будет либо 1 гигантский вектор (бесполезно), либо потеря логики в обрывках текста.

In [16]:
# ✅ Разбиение на чанки (Nodes)
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)
nodes = splitter.get_nodes_from_documents(docs)

📦 Document

Это исходный файл или текст, который ты загружаешь (PDF, DOCX, письмо, страница сайта и т.д.)

docs = SimpleDirectoryReader("data").load_data()  # -> List[Document]
🧠 В нём может быть:

один большой блок текста

метаданные (название файла, дата, категория и т.п.)

может весить тысячи токенов

🧩 Node

Это фрагмент документа — результат разбиения (splitter), который уже готов к векторизации и поиску.

splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(docs)  # -> List[TextNode]

Каждый Node:

содержит небольшой кусок текста (чанк)

знает, из какого документа он пришёл

может иметь собственные метаданные, ID, позиции и т.д.

🧠 Запомни правило:

Document → это "сырой текст"

Node → это "боевой юнит" для RAG

Нужно построить индекс? ➝ Работай с Node.

In [17]:
# ✅ Построение индекса и движка запросов
index = VectorStoreIndex(nodes)
engine = index.as_query_engine()

In [19]:
# ✅ Пример запроса
response = engine.query("How to quantify the Issue?")
Markdown(response.response)

To quantify the issue, integrate compelling data points that validate the challenges faced by your prospects. This data not only substantiates the problem but also places it within a broader context, potentially underscoring the urgency of addressing the issue. By effectively communicating the significance of the problem and the potential repercussions of ignoring it, you can emphasize the need for a prompt and effective solution.

# Пример запроса
response = engine.query("Что можно предложить потенциальному клиенту на этапе первого контакта?")
Markdown(response.response)

In [21]:
for node in nodes:
    text = node.get_content().lower()
    if any(kw.lower() in text for kw in [
        "comparative statistics",
        "framing your product's capabilities",
        "personalize your email",
        "use a specific cta"
    ]):
        print(text[:300], "\n---")

4
hi [first name],
i enjoyed our call today, and i hope you 
did too. here are the top value adds we 
went over: 
next steps: 
i’ll [next step] so we can proceed to 
[shared goal].
@[first name]: you mentioned 
needing to check [item], would it be 
possible to do so before [next step] 
so i can show 
---
6
6. multi-threading: looping the decision 
maker back in
hi [first name],
as an update, the team feedback has 
been very positive and we’ve surfaced 
great use cases and potential results.
we’re now preparing for an executive 
presentation, and we’d like to loop 
you in with you to showcase the 
in 
---
7
hi [first name],
curious if i could put some time on 
your calendar ahead of our executive 
presentation on [date/time]?
want to make sure we’re addressing all 
stakeholders in the call and surfacing 
any concerns the team has ahead of 
[next step for buyers].
does [date/time] work?
- [your name]
 
---
8
hi [first name],
totally get where you’re coming from.
let’s hop on a quick call

## 🧪 Практика 2: Сравнение ответов
Запустим серию запросов для разных документов и посмотрим, какие фрагменты выбирает бот.

In [23]:
# TODO: Несколько запросов
queries = [
    "How to close a deal?",
    "In what cases it is suitable to follow up email ?",
    "How to show benefits of your product to customer?"
]
for q in queries:
    print(f"🧠 Запрос: {q}")
    print(engine.query(q).response, "\n")

🧠 Запрос: How to close a deal?
To close a deal effectively, it is crucial to only send your proposal after scheduling a call to walk through it with your buyers. This allows you to control the conversation, understand how your proposal is being received, and identify any potential hurdles. Additionally, take control of the next steps by gathering as much information as possible to address any concerns and ensure a smooth closing process. Proceed with caution during this crucial step to avoid jeopardizing the deal after putting in significant effort. 

🧠 Запрос: In what cases it is suitable to follow up email ?
It is suitable to follow up with an email when you want to keep the message simple, provide useful insights even if the recipient doesn't buy from you, tell a story that resonates with the prospect's experience, and include a specific interest call-to-action to continue the conversation. 

🧠 Запрос: How to show benefits of your product to customer?
Showcasing testimonials from sa

🧠 Зачем это нужно:

Ты строишь бота для продаж, и хочешь:

автоматом находить в презентациях и письмах фразы о выгодах

передавать их команде → например, в виде таблицы для CRM или маркетолога

📌 Это и делает этот экспорт: выделяет “выгоды” → сохраняет в таблицу.

In [24]:
# TODO: Экспорт в CSV
import pandas as pd

relevant = [
    node.get_content()
    for node in nodes
    if "выгода" in node.get_content().lower()
]

df = pd.DataFrame(relevant, columns=["Fragment"])
df.to_csv("extracted_sales_benefits.csv", index=False)


# 🔁 Part 2: LangGraph Loop для уточнения запроса

**Цель:** если ответ короткий или нерелевантный, повторно задать вопрос — уточнённо.
Используем LangGraph для построения простого цикла на 2 узла:
- `query_node`: делает запрос
- `refine_node`: проверяет длину → при необходимости переформулирует

In [26]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.4 MB/s eta 0:00:00


In [31]:
from langgraph.graph import StateGraph
from typing import TypedDict
from langchain.schema import BaseMessage

# Структура состояния
class State(TypedDict):
    question: str
    history: list[str]
    last_answer: str

# Узел 1 — делает запрос
def query_node(state: State) -> State:
    answer = engine.query(state["question"]).response.strip()
    state["history"].append(answer)
    state["last_answer"] = answer
    return state

# Узел 2 — проверяет длину ответа
def refine_node(state: State) -> str:
    if len(state["last_answer"].split()) < 20:
        state["question"] = f"Попробуй точнее: {state['question']}"
        return "query"
    return "end"

# Добавим фиктивный end-узел
def end_node(state: State) -> State:
    return state  # просто возвращает финальное состояние



📌 В LangGraph ты должен добавить все узлы, даже "end". Он не создаётся автоматически.



In [33]:
# Сборка графа
graph = StateGraph(State)
graph.add_node("query", query_node)
graph.add_node("end", end_node)  # обязательно!
graph.add_conditional_edges("query", refine_node)
graph.set_entry_point("query")
graph.set_finish_point("end")
app = graph.compile()

# Запуск
initial_state = {
    "question": "Что предложить клиенту?",
    "history": [],
    "last_answer": ""
}

final_state = app.invoke(initial_state)

# Вывод
print("🔁 История итераций:\n")
for i, step in enumerate(final_state["history"], 1):
    print(f"{i}) {step}\n")



🔁 История итераций:

1) Offer testimonials from satisfied customers, present compelling statistics showcasing product effectiveness, provide a comparative analysis of the market to highlight product differentiation, include case studies or success stories illustrating impact on similar businesses, discuss industry benchmarks, and demonstrate how the product helps clients exceed standards. Additionally, highlight specific, quantifiable improvements that the client can expect upon integrating the product, offer predictions on improvement degrees for key performance indicators, and provide a realistic timeline for when these benefits are likely to manifest.



🧱 class State(TypedDict)
Это формат состояния, которое «течёт» по графу.

class State(TypedDict):
    question: str
    history: list[str]
question: текущий текст запроса

history: список всех ответов бота

last_answer: последний ответ, чтобы проверить его длину

🔄 def query_node(...)
Этот узел делает основной запрос:

answer = engine.query(state["question"]).response.strip()
Сохраняет ответ в history

Обновляет last_answer

🔁 def refine_node(...)
Решает, нужно ли уточнение:


if len(state["last_answer"].split()) < 20:
    state["question"] = f"Попробуй точнее: {state['question']}"
    return "query"  # Цикл
return "end"
Если ответ короче 20 слов → считаем его слабым → переформулируем вопрос → возвращаемся в query

Иначе — завершаем

🎯 Результат:
Если бот даёт "пустой" или общий ответ — LangGraph автоматически уточняет вопрос и повторяет.

